# 네이버 뉴스 기사 스크래핑 

In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import requests
import pandas as pd 

In [ ]:
def get_news(query, page_num=10):
    
    # 저장할 데이터 프레임을 만든다. 
    news_df = pd.DataFrame(columns=('Title','Link','Press','Datetime','Article'))
    ind = 0
    
    # 파라미터로 질의 언어를 받아옴 
    url_query = quote(query) # 받은 질의문을 utf-8으로 변환 
    url = "https://search.naver.com/search.naver?&where=news&sm=nws_hty&query=" + url_query
    
   
    for _ in range(0,page_num):
            
        # search_url = urllib.request.urlopen(url,headers={'User-Agent':'Mozilla/5.0'}).read() # 해당 유알엘을 읽기모드로 여는듯
        search_url = requests.get(url,headers ={'User-Agent':'Mozilla/5.0'})
        soup = BeautifulSoup(search_url.text,'html.parser') # 뷰숩으로 탐색기 열어주는듯
    
        links = soup.find_all('div',{'class':'info_group'})

 
        for link in links: # 해당 테그를 가진 것들을 모두 들고와 반복작업 
            #print("*********link*******"+str(link))
            # 발행 언론사 정보(글씨)를 가져옵니다. 
            press = link.find('a',{'class':'info press'}).get_text()
            #press = link.find('a',{'class':'info press'}).find('span').get_text()


            news_url = link.find('a',{'class':'info'}).get('href') # a테그를 가지면 네이버 뉴스 테그가 심어진것임
            # 이때 그걸 가진것에서 --> href :: 뉴스 링크를 가져옴 

            if (news_url == '#'): # 뉴스 유알엘이 없는경우 
                print('url없음!',end="")
                continue
                # 넘어가 버림 
            else: 
                # 링크가 있는경우 누르고, 내용 가져오게 

                # 링크 타고 이동  
                news_link = requests.get(news_url,headers ={'User-Agent':'Mozilla/5.0'})
                news_html = BeautifulSoup(news_link.text,'html.parser')
                # print 
                #print(news_html)
                #print("\n\n\nn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")
                try:
                    # 제목 가져오기 
                    title = news_html.find('h3',{'id':'articleTitle'}).get_text()
                    #print("TITLE*********:"+ title)

                    # 기사 날짜 담아오기 
                    datetime = news_html.find('span',{'class':'t11'}).get_text()

                    # 기사 본문 가져오기 , 텍스트만 
                    article = news_html.find('div',{'id':'articleBodyContents'}).get_text()

                    news_df.loc[ind] = [title, news_url,press,datetime,article]
                    ind += 1
                    print("#",end="")
                except:
                    print('!',end="")
                    continue
        try:
            next_page = soup.find('a',{'class':'btn_next'}).get('href')
            url = "https://search.naver.com/search.naver" + next_page 
        except:
            print("@")
            break
        
    return news_df
    

In [ ]:
import requests
from bs4 import BeautifulSoup

# 언론사 , 작성일, 제목, 본문

def get_news(url):
    result = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(result.text, "html.parser")
    press = soup.find('div',{'class':'article_header'}).find('img')['title']
    date = soup.find('div',{'class':'sponsor'}).find('span',{'class':'t11'}).get_text();
    title = soup.find('h3',{'id':'articleTitle'}).get_text();
    before_content = soup.find('div',{'id':'articleBodyContents'});

    content = ""
    for item in before_content.contents:
        stripped = str(item).strip()
        if stripped == "":
            continue
        if stripped[0] not in ["<", "/"]:
            content += str(item).strip()
    content = content.replace("&apos;", "")
    content = content.replace("본문 내용TV플레이어", "")
    print(f'언론사: {press}\n작성일: {date}\n기사제목: {title}\n본문: {content}\n\n')

user_input = input("키워드: ")
page_number = int(input("찾을 페이지 수: "))
print()

for number in range(page_number):
    print(f'***** {number+1}페이지 탐색 중 *****')
    number = (number*10)+1
    url = f'https://search.naver.com/search.naver?query={user_input}&where=news&start={number}'
    result = requests.get(url)
    soup = BeautifulSoup(result.text, "html.parser")
    info_group = soup.find_all('div',{'class':"info_group"})
    for info in info_group:
        try:
            news_url = info.find_all('a')[1]['href']
            get_news(news_url)
        except:
            pass

# 성공 버전!

In [43]:
from bs4 import BeautifulSoup  # 접속후 웹스크립트에 대한 접근을 위함
from urllib.parse import quote  # 입력어를 인코딩
import requests  # 페이지에 접근
import pandas as pd  # 결과 저장


def get_news(query="", page_num=10, ds="2021.05.03", de="2021.05.03"):
    result_df = pd.DataFrame(columns=('title', 'news_url', 'press', 'date', 'content'))
    ind = 0

    # 시작날짜, 메인함수에서 str(yyyy.mm.dd) 형식으로 받기!
    # ds
    # de

    #print(type(query))
    url = f'https://search.naver.com/search.naver?&where=news&sm=nws_hty&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={ds}&de={de}&query={query}'
    # sort=2 옵션은 오래된 순 정렬
    #print(url)

    while (True):

        # 일단 해당 유알엘에 접속후 뷰리플숩으로 스크립트 열기
        search_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(search_url.text, "html.parser")

        links = soup.find_all('div', {'class': 'news_area'})

        for link in links:
            ## 제목 검사
            ## 통과한것만 infogroup 가져오기
            ## 탐색
            title = link.find('a', {'class': 'news_tit'})['title'];
            #print(f'title: {title}\nquery in?: {query in title}')

            if (query in title):
                info_group = link.find_all('div', {'class': "info_group"})
                for info in info_group:

                    try:
                        news_url = info.find_all('a')[1]['href']
                        press, date, content = get_contents(news_url)
                        result_df.loc[ind] = [title, news_url, press, date, content]
                        ind += 1
                        print("#", end="")
                    except:
                        print('!', end="")
                        pass
            else:
                print("$", end="")
                pass

        print("\n==========================\n")

        try:
           next_page = soup.find('a', {'class': 'btn_next'}).get('href')
           url = "https://search.naver.com/search.naver" + next_page
        except:
           print("\n")
           print("--전체 페이지 확인 끝--")
           break
    return result_df


def get_contents(news_url):
    if (news_url == '#'):  # 뉴스 유알엘이 없는경우
        print('url없음!', end="")
        pass
        # 넘어가 버림
    else:
        # 링크가 있는경우 누르고, 내용 가져오게

        # 링크 타고 이동
        news_link = requests.get(news_url, headers={'User-Agent': 'Mozilla/5.0'})
        news_html = BeautifulSoup(news_link.text, 'html.parser')
        press = news_html.find('div', {'class': 'article_header'}).find('img')['title']
        date = news_html.find('div', {'class': 'sponsor'}).find('span', {'class': 't11'}).get_text();

        before_content = news_html.find('div', {'id': 'articleBodyContents'});

        content = ""
        for item in before_content.contents:
            stripped = str(item).strip()
            if stripped == "":
                continue
            if stripped[0] not in ["<", "/"]:
                content += str(item).strip()
        content = content.replace("&apos;", "")
        content = content.replace("본문 내용TV플레이어", "")

        return press, date, content



In [44]:
user_input = input("키워드: ")
df = get_news(query=user_input)
print(df)

키워드: 카카오톡
$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$!$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$##$$$

$!$$$$$!!$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

#$$$$$$!$$

$$$!$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$!$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$!$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$!$$$$

$$$$$$$$$$

$$$$$!$$$$

$$$$$$#$$$

$$$$$$$$$$

$$$$$$$$$$

$$$!!$$$$$

$$$$$$$!$$

$$$$$$$$$$

$$$$$$$$$!

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$#$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$#$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$!$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$!$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$$$$

$$$$$$$



--전체 페이지 확인 끝--
                                     title  \
0              3명에게 '카카오톡 선물' 하면 5000원 받는다   
1      

In [45]:
df.to_excel('test_crawling.xlsx')